## Load Data
#### Data was processed into a cubes in TensorFlow notebook
#### See notebooks from Udacity TensorFlow mooc

In [1]:
from six.moves import cPickle as pickle

In [2]:
loaded = pickle.load( open('../data/notMNIST.pickle', 'rb') )
loaded.keys()

['train_labels',
 'valid_labels',
 'test_labels',
 'test_dataset',
 'train_dataset',
 'valid_dataset']

In [3]:
print(loaded['train_dataset'].shape)
print(loaded['test_dataset'].shape)
print(loaded['valid_dataset'].shape)

(200000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)


In [6]:
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled)
Using Theano backend.


In [7]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [9]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [44]:
X_train = loaded['train_dataset']
y_train = onehot(loaded['train_labels'])

X_valid = loaded['valid_dataset']
y_valid = onehot(loaded['valid_labels'])

X_test = loaded['test_dataset']
y_test = onehot(loaded['test_labels'])

In [46]:
y_train.shape

(200000, 10)

In [47]:
X_valid = np.expand_dims(X_valid,1)
X_train = np.expand_dims(X_train,1)
X_test = np.expand_dims(X_test,1)

In [48]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

def norm_input(x):
    return (x-mean_px)/std_px

## Convolution Network

In [49]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [73]:
model = get_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [74]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 1, 28, 28)     0           lambda_input_5[0][0]             
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 32, 26, 26)    320         lambda_5[0][0]                   
____________________________________________________________________________________________________
convolution2d_18 (Convolution2D) (None, 32, 24, 24)    9248        convolution2d_17[0][0]           
____________________________________________________________________________________________________
maxpooling2d_9 (MaxPooling2D)    (None, 32, 12, 12)    0           convolution2d_18[0][0]           
___________________________________________________________________________________________

In [75]:
batch_size = 64
#
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=batch_size)
test_batches = gen.flow(X_test, y_test, batch_size=batch_size)

In [76]:
model.optimizer.lr = 0.0001

In [77]:
model.fit_generator(batches, min(batch_size*100,batches.n), nb_epoch=10, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-3.13--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.11-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /home/ubuntu/.theano/compiledir_Linux-3.13--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.11-64/lock_dir/lock


Epoch 1/10
6400/6400 [==============================] - 3s - loss: 1.5547 - acc: 0.5484 - val_loss: 0.7789 - val_acc: 0.7689
Epoch 2/10
6400/6400 [==============================] - 3s - loss: 0.8852 - acc: 0.7573 - val_loss: 0.5748 - val_acc: 0.8521
Epoch 3/10
6400/6400 [==============================] - 3s - loss: 0.7829 - acc: 0.7844 - val_loss: 0.4990 - val_acc: 0.8628
Epoch 4/10
6400/6400 [==============================] - 3s - loss: 0.6961 - acc: 0.8064 - val_loss: 0.4289 - val_acc: 0.8822
Epoch 5/10
6400/6400 [==============================] - 3s - loss: 0.6631 - acc: 0.8117 - val_loss: 0.3927 - val_acc: 0.8934
Epoch 6/10
6400/6400 [==============================] - 3s - loss: 0.6086 - acc: 0.8297 - val_loss: 0.3664 - val_acc: 0.9000
Epoch 7/10
6400/6400 [==============================] - 3s - loss: 0.5732 - acc: 0.8358 - val_loss: 0.3469 - val_acc: 0.9035
Epoch 8/10
6400/6400 [==============================] - 3s - loss: 0.5465 - acc: 0.8406 - val_loss: 0.3286 - val_acc: 0.9082
